In [7]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
from importlib import reload
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
from functools import reduce
import open3d as o3d
import random
import glob
import os
import pandas as pd
from collections import defaultdict
import datetime
from csv import writer
import traceback
import concurrent.futures

In [8]:
def random_color():
    """Generate a random RGB color."""
    return [random.random(), random.random(), random.random()]

def open3d_block_by_element(ifc_file): 
    # Configuration for ifcopenshell geometric settings
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)

    # Initialize the iterator for geometric representations
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    
    # Define entity types to include and exclude
    exclude_list = set(["IfcCovering", "IfcFurnishingElement", "IfcSpace", "IfcOpening", "IfcOpeningElement", "IfcRailing"])
    
    # Lists to store the resulting meshes and element information
    all_meshes = []
    element_information = {}
    dims = []

    # Variable to track the smallest dimension among all meshes
    temp_dimension_x = 0.8
    temp_dimension_y = 0.8
    temp_dimension_z = 0.8
    
    # Iterate over the geometric representations
    if iterator.initialize():
        index = 0
        while True:
            shape = iterator.get()
            
            # Check if the shape type is not excluded and represents an IfcBuildingElement
            if shape.type not in exclude_list and shape.product.is_a("IfcBuildingElement"):
                # Convert the shape's geometry to an Open3D mesh
                faces = np.array(shape.geometry.faces)
                verts = np.array(shape.geometry.verts).reshape(-1, 3)
                mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(verts),
                                                 triangles=o3d.utility.Vector3iVector(faces.reshape(-1, 3)))
                
                # Update the smallest dimension if necessary
                dimensions = mesh.get_max_bound() - mesh.get_min_bound()
                dims.append(dimensions)
                min_dimension_x, min_dimension_y, min_dimension_z = dimensions

                # Update temp_dimension_x, temp_dimension_y, temp_dimension_z if necessary
                if min(temp_dimension_x, min_dimension_x) > 0.1:
                    temp_dimension_x = min(temp_dimension_x, min_dimension_x)
                if min(temp_dimension_y, min_dimension_y) > 0.1:
                    temp_dimension_y = min(temp_dimension_y, min_dimension_y)

                # Store the mesh and its corresponding IFC GUID
                all_meshes.append(mesh)
                element_information[index] = shape.guid
                index += 1
                
                # Check if the shape is a slab and update the maximum x dimension if necessary
                if shape.type == "IfcSlab":
                    if min(temp_dimension_z, min_dimension_z) >= 0.1:
                        temp_dimension_z = min(temp_dimension_z, min_dimension_z)/2                               
                    
            # Move to the next geometric representation
            if not iterator.next():
                
                break

    return all_meshes, element_information, round(temp_dimension_x,2), round(temp_dimension_y,2), round(temp_dimension_z,2), dims

def create_uniform_grid(bounds, voxel_size_x, voxel_size_y, voxel_size_z):
    """Create a uniform grid within the given bounds with rectangular cells."""
    x = np.arange(bounds[0], bounds[1] + voxel_size_x, voxel_size_x)
    y = np.arange(bounds[2], bounds[3] + voxel_size_y, voxel_size_y)
    z = np.arange(bounds[4], bounds[5] + voxel_size_z, voxel_size_z)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def open3d_to_pyvista(point_cloud_o3d):
    """
    Convert an Open3D point cloud to a PyVista point cloud.

    Parameters:
    - point_cloud_o3d: The Open3D point cloud.

    Returns:
    - A PyVista `PolyData` object.
    """
    # Extract points from Open3D point cloud
    points = np.asarray(point_cloud_o3d.points)

    # Create a PyVista PolyData object
    point_cloud_pv = pv.PolyData(points)

    return point_cloud_pv

def process_point(pcd, mesh_guid_index, grid_bounds, x_size, y_size, z_size, dims, grid_attributes):
    j = int((pcd[0] - grid_bounds[0]) / x_size)
    i = int((pcd[1] - grid_bounds[2]) / y_size)
    k = int((pcd[2] - grid_bounds[4]) / z_size)
    
    if 0 <= i < dims[0]-1 and 0 <= j < dims[1]-1 and 0 <= k < dims[2]-1 and grid_attributes[i,j,k] == -1:
        grid_attributes[i,j,k] = mesh_guid_index

def voxelize_space(bounds, pcd_list, x_size, y_size, z_size, offset_x, offset_y, offset_z):
    """Create a 3d grid and check the intersections of the meshes with the grid, with offsets."""
    # Adjust bounds with the offsets
    adjusted_bounds = [
        bounds[0] + offset_x, bounds[1] - offset_x,
        bounds[2] + offset_y, bounds[3] - offset_y,
        bounds[4] + offset_z, bounds[5] - offset_z
    ]

    grid = create_uniform_grid(adjusted_bounds, x_size, y_size, z_size)
    
    # create an empty 3d array in the same dimensions of the grid
    dims = grid.dimensions
    grid_attributes = np.full((dims[0]-1, dims[1]-1, dims[2]-1), -1, dtype=int)

    grid_bounds = grid.bounds

    [process_point(point, i, grid_bounds, x_size, y_size, z_size, dims, grid_attributes) 
    for i, pcd in enumerate(pcd_list) for point in pcd.points]
 
    # assign empty array to the grid
    grid.cell_data['attributes'] = grid_attributes.flatten(order='F').astype(int)
    
    return grid
    
def get_sampling_points(mesh, voxel_size, points_per_unit_area=120):
    
    # Calculate the surface area of the mesh
    area = mesh.get_surface_area()

    # Calculate the number of points to sample
    N = int(area * points_per_unit_area*1/voxel_size)
        
    return N

def create_point_cloud(all_meshes, x_size):
    """Create a point cloud from a mesh."""

    pcd_list = []

    for mesh in all_meshes:
        # Get the number of points to sample based on the mesh size
        N = get_sampling_points(mesh, x_size)
        pcd = mesh.sample_points_uniformly(N)
        pcd_pv = open3d_to_pyvista(pcd)
        pcd_list.append(pcd_pv)  
        
    return pcd_list


def fetch_element_quantities(ifc_file, grid, element_information):
    # Fetch all property sets named 'ZECH_ATTRIBUTE'
    start_time = time.time()
    print(f'Calculating quantities in {ifc_file}...')
    property_sets = ifc_file.by_type("IfcPropertySet")
    zech_property_sets = [ps for ps in property_sets if ps.Name == "ZECH_ATTRIBUTE"]
    
    # Create a mapping of elements to their ZB_AUSWAHL values
    element_to_zb_auswahl = {}
    for ps in zech_property_sets:
        for prop in ps.HasProperties:
            if prop.is_a("IfcPropertySingleValue") and prop.Name == "ZB_AUSWAHL":
                # Find all elements that have this property set
                for rel in ifc_file.by_type("IfcRelDefinesByProperties"):
                    if rel.RelatingPropertyDefinition == ps:
                        for related_object in rel.RelatedObjects:
                            element_to_zb_auswahl[related_object.GlobalId] = str(prop.NominalValue.wrappedValue)
    
    element_counts = defaultdict(int)

    # Calculate counts and collect x, y, z values for cells matching the attribute
    for i, att in enumerate(grid.cell_data['attributes']):
        element_guid = element_information.get(att)
        zb_auswahl_value = element_to_zb_auswahl.get(element_guid)
        if zb_auswahl_value is not None:
            element_counts[zb_auswahl_value] += 1

    results_QTO = {}
     # Create a mapping of ZB_AUSWAHL values to their building element types
    zb_auswahl_to_element_type = {}
    for element_guid, zb_auswahl_value in element_to_zb_auswahl.items():
        element = ifc_file.by_guid(element_guid)
        if element:
            element_type = element.is_a()
            zb_auswahl_to_element_type[zb_auswahl_value] = element_type

    # Store the counts for different ZB_AUSWAHL values
    for zb_auswahl_value, count in element_counts.items():
        results_QTO[f'{zb_auswahl_value}_count'] = count

    end_time = time.time()
    quantity_calculation_time = end_time - start_time
    results_QTO['Quantity Calculation (sec)'] = round(quantity_calculation_time,2)
    results_QTO['ZB_AUSWAHL_to_Element_Type'] = zb_auswahl_to_element_type

    return results_QTO

def process_ifc_file(file_name, x_size, y_size, z_size, x_offset, y_offset, z_offset):
    try:
        
        pv.set_plot_theme("document")
        pv.global_theme.window_size = [1024, 768]
        pv.global_theme.jupyter_backend = 'static'
        p = pv.Plotter()
        ifc_file = ifcopenshell.open(file_name)
        print(file_name)

        ### CONVERSION FROM IFC TO MESH ###
        print(f'Converting {file_name}...')

        start_time = time.time()
        all_meshes, element_information, x,y,z, dims = open3d_block_by_element(ifc_file)

        total_polygons = sum(len(mesh.triangles) for mesh in all_meshes)

        # combine all meshes into one mesh and find the bounds
        combined_mesh = reduce(lambda m1, m2: m1 + m2, all_meshes)

        end_time = time.time()
        conversion_time = end_time - start_time

        ### POINT CLOUD CREATION ###
        print(f'Point cloud formation representing {file_name}...')
        start_time = time.time()
        point_cloud_list  = create_point_cloud(all_meshes, x_size)
        
        #merge all the list of point clouds into one
        pcd_pv = pv.PolyData()
        pcd_pv.points = np.vstack(list(map(lambda x: x.points, point_cloud_list)))
        sampling_points = len(pcd_pv.points)
        end_time = time.time()
        point_cloud_processing = end_time - start_time

        print(f'Rasterizing {file_name}...')
        start_time = time.time()

        # Compute the oriented bounding box (OBB)
        bb = combined_mesh.get_axis_aligned_bounding_box()
        
        # Extract the eight corner points of the OBB
        bb_points = np.asarray(bb.get_box_points())

        # Compute the axis-aligned bounding box of these eight points
        xmin, ymin, zmin = np.min(bb_points, axis=0)
        xmax, ymax, zmax = np.max(bb_points, axis=0)

        bounds = np.array([xmin, xmax, ymin, ymax, zmin, zmax])
        
        ### RASTERIZATION ###
        
        grid = voxelize_space(bounds, point_cloud_list, x_size, y_size, z_size, x_offset, y_offset, z_offset)
        end_time = time.time()
        rasterization_time = end_time - start_time
        
        end_time = time.time()
        rasterization_time = end_time - start_time        

        ### VISUALIZATION ###
        print(f'Visualizing {file_name}...')
        start_time = time.time()
         
        # Extract the cells with non-empty attribute values
        non_empty_cells = grid.cell_data['attributes'] != -1  
        # Create a box from the bounds
        box = pv.Box(bounds=bounds)

        # Add the box to the plotter
        p.add_mesh(box, color="blue", style="wireframe", line_width=2)
        # Add the mesh to the plotter with the non-empty cells
        p.add_mesh(grid.extract_cells(non_empty_cells), opacity=0.8,show_edges=False,show_scalar_bar=False)
        
        screenshot_filename = os.path.join("screenshots", f"{file_name}.png")

        p.screenshot(screenshot_filename)
        end_time = time.time()
        visualization_time = end_time - start_time
        #p.show()
         # Store the results in a dictionary
        results = {}
        results['File'] = file_name
        results['# of Polygons'] = total_polygons
        results['# of Points'] = sampling_points
        results['Cell Size (m)'] = x_size, y_size, z_size
        results['Conversion (sec)'] = round(conversion_time,2)
        results['Point Cloud Creation (sec)'] = round(point_cloud_processing,2)
        results['Rasterization (sec)'] = round(rasterization_time,2)
        results['Visualization (sec)'] = round(visualization_time,2)
        
        element_counts = fetch_element_quantities(ifc_file, grid, element_information)
        '''

        # Write results to Excel sheets
        if writer:
            # Write to 'Results' sheet
            results_df = pd.DataFrame([results])
            results_df.to_excel(writer, sheet_name='Results', startrow=writer.sheets['Results'].max_row, index=False, header=False)
            
            # Write to 'QTO' sheet
            element_counts_df = pd.DataFrame([element_counts])
            element_counts_df.to_excel(writer, sheet_name='QTO', startrow=writer.sheets['QTO'].max_row, index=False, header=False)'''

        return results, element_counts

    except FileNotFoundError:
        print(f"Error: {file_name} not found.")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}

    except MemoryError:
        print(f"Error: Insufficient memory to process {file_name}.")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}

    except (TypeError, ValueError) as e:
        print(f"Error processing {file_name}: {str(e)}")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}

    except Exception as e:
        print(f"An unexpected error occurred while processing {file_name}: {str(e)}")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}
    
def find_ifc_files(directory):
    """Find all IFC files in a given directory."""
    # Use os.path.join to ensure the path is constructed correctly for any OS
    search_path = os.path.join(directory, "*.ifc")
    return glob.glob(search_path)

In [9]:


file_name = "IFC Files/05.ifc"

cell_size = 0.1
# Generate offsets as fractions of the cell size
offsets = [cell_size * i for i in [0.25, 0.5, 0.75, 0.0]]

# Generate all possible combinations of offsets
parameter_sets = [
    {'x_offset': x, 'y_offset': y, 'z_offset': z}
    for x in offsets
    for y in offsets
    for z in offsets
]

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string: YYYYMMDD_HHMMSS
formatted_time = now.strftime("%Y%m%d_%H%M%S")

# Create a filename using the formatted time
filename = f"output_{formatted_time}_offset.txt"

# Open the file once and write results for each parameter set
with open(filename, 'a') as f:
    for param in parameter_sets:
        print(f"Processing with offset: {param['x_offset'], param['y_offset'], param['z_offset']}")
        # Call the process_ifc_file function with the current offset
        # Assuming process_ifc_file is a function you have defined elsewhere
        results1, results2 = process_ifc_file(
            file_name, cell_size, cell_size, cell_size, param['x_offset'], param['y_offset'], param['z_offset'])

        # Convert the dictionaries to lists of strings
        results1_list = [f"{key} : {value}" for key, value in results1.items()]
        results2_list = [f"{key} : {value}" for key, value in results2.items()]
        print(results1, results2)
        
        # Append results to the text file
        f.write(f"Offset: {param['x_offset']}, {param['y_offset']}, {param['z_offset']}\n")
        f.write(','.join(results1_list) + '\n')
        f.write(','.join(results2_list) + '\n')


Processing with offset: (0.025, 0.025, 0.025)
IFC Files/05.ifc
Converting IFC Files/05.ifc...
Point cloud formation representing IFC Files/05.ifc...
Rasterizing IFC Files/05.ifc...
Visualizing IFC Files/05.ifc...
Calculating quantities in <ifcopenshell.file.file object at 0x0000020E5C0EE530>...
{'File': 'IFC Files/05.ifc', '# of Polygons': 39244, '# of Points': 64165466, 'Cell Size (m)': (0.1, 0.1, 0.1), 'Conversion (sec)': 4.27, 'Point Cloud Creation (sec)': 10.12, 'Rasterization (sec)': 282.87, 'Visualization (sec)': 6.8} {'ZB_PERI_W_count': 117006, 'ZB_SOHLE_count': 437113, 'ZB_UNTF_count': 3158, 'ZB_STB_UZ_count': 208653, 'ZB_STR_FUN_count': 111403, 'ZB_ORT_UGW_count': 127802, 'ZB_RECK_ST_count': 182713, 'ZB_STB_ÜZ_count': 288888, 'ZB_ORT_W_count': 647766, 'ZB_Z_MW_count': 168582, 'ZB_FT_TRH_count': 54736, 'ZB_FT_POD_count': 24845, 'ZB_P_FUN_count': 6792, 'ZB_ORT_DECKE_count': 2462388, 'ZB_RU_ST_count': 5914, 'ZB_PERI_BO_count': 3, 'ZB_ORT_WRU_count': 1318, 'ZB_STB_ATT_count': 5509

ERROR:root:Unable to allocate 266519214 elements of size 8 bytes. 


: 

In [6]:
cell_sizes = [round(i, 2) for i in list(np.arange(0.1, 0.11, 0.1))]
cell_sizes = sorted(cell_sizes, reverse=True)  
parameter_sets = [
    {'x_size': x, 'y_size': x, 'z_size': z}
    for x in cell_sizes
    for z in cell_sizes
]

file_name = "IFC Files/05.ifc"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string: YYYYMMDD_HHMMSS
formatted_time = now.strftime("%Y%m%d")

# Create a filename using the formatted time
filename = f"output_{formatted_time}_affine_testing.txt"

# Open the file once and write results for each parameter set
with open(filename, 'a') as f:
    # Process the IFC file with each set of parameters and write the results to the text file
    for params in parameter_sets:
        results1, results2 = process_ifc_file(file_name, params['x_size'], params['y_size'], params['z_size'])

        # Convert the dictionaries to lists of strings
        results1_list = [str(res1) + " : " + str(results1.get(res1)) for res1 in results1]
        results2_list = [str(res2) + " : " + str(results1.get(res2)) for res2 in results2]
        print(results1, results2)
        # Append results to the text file
        f.write(','.join(results1_list) + '\n')
        f.write(','.join(results2_list) + '\n')


IFC Files\01_Test.ifc
Converting IFC Files\01_Test.ifc...
Point cloud formation representing IFC Files\01_Test.ifc...
Rasterizing IFC Files\01_Test.ifc...
Visualizing IFC Files\01_Test.ifc...
Calculating quantities in <ifcopenshell.file.file object at 0x0000026D6AA4FB80>...
{'File': 'IFC Files\\01_Test.ifc', '# of Polygons': 1056, '# of Points': 142941, 'Cell Size (m)': (0.1, 0.1, 0.1), 'Conversion (sec)': 0.09, 'Point Cloud Creation (sec)': 0.02, 'Rasterization (sec)': 0.57, 'Visualization (sec)': 0.2} {'Quantity Calculation (sec)': 0.02, 'ZB_AUSWAHL_to_Element_Type': {}}
IFC Files\02_Duplex.ifc
Converting IFC Files\02_Duplex.ifc...
Point cloud formation representing IFC Files\02_Duplex.ifc...
Rasterizing IFC Files\02_Duplex.ifc...
Visualizing IFC Files\02_Duplex.ifc...
Calculating quantities in <ifcopenshell.file.file object at 0x0000026D12B5DA80>...
{'File': 'IFC Files\\02_Duplex.ifc', '# of Polygons': 9420, '# of Points': 4318024, 'Cell Size (m)': (0.1, 0.1, 0.1), 'Conversion (sec)

In [5]:
from collections import deque

def is_valid_cell(i, j, k, dims, grid_attributes, visited):
    return 0 <= i < dims[0]-1 and 0 <= j < dims[1]-1 and 0 <= k < dims[2]-1 and \
           grid_attributes[i, j, k] == -1 and not visited[i, j, k]

def flood_fill(grid_attributes, start_cell):
    dims = grid_attributes.shape
    visited = np.zeros_like(grid_attributes, dtype=bool)
    rooms = []

    # Define possible moves (dx, dy, dz)
    moves = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]

    # Initialize queue for BFS
    queue = deque([start_cell])

    while queue:
        i, j, k = queue.popleft()

        if not visited[i, j, k]:
            visited[i, j, k] = True

            # Check if this cell is part of a room
            if grid_attributes[i, j, k] == -1:
                rooms.append((i, j, k))

            # Explore neighbors
            for dx, dy, dz in moves:
                ni, nj, nk = i + dx, j + dy, k + dz
                if is_valid_cell(ni, nj, nk, dims, grid_attributes, visited):
                    queue.append((ni, nj, nk))

    return rooms

# Find the first filled cell
start_cell = None
grid_attributes_3d = grid.cell_data['attributes'].reshape(grid.dimensions[0]-1, grid.dimensions[1]-1, grid.dimensions[2]-1, order='F')

for i in range(grid.dimensions[0]-1):
    for j in range(grid.dimensions[1]-1):
        for k in range(grid.dimensions[2]-1):
            if grid_attributes_3d[i, j, k] != -1:
                start_cell = (i, j, k)
                break
    if start_cell:
        break

# Find the rooms
if start_cell:
    rooms = flood_fill(grid_attributes_3d, start_cell)
    print("Found rooms:", rooms)
else:
    print("No filled cell found.")


Found rooms: [(0, 13, 2), (0, 12, 1), (1, 13, 2), (0, 14, 2), (0, 13, 3), (0, 13, 1), (1, 12, 1), (0, 11, 1), (0, 12, 0), (2, 13, 2), (1, 14, 2), (1, 13, 3), (1, 13, 1), (0, 15, 2), (0, 14, 3), (0, 14, 1), (0, 13, 4), (0, 13, 0), (2, 12, 1), (1, 11, 1), (1, 12, 0), (0, 10, 1), (0, 11, 0), (3, 13, 2), (2, 14, 2), (2, 13, 3), (2, 13, 1), (1, 15, 2), (1, 14, 3), (1, 14, 1), (1, 13, 4), (1, 13, 0), (0, 16, 2), (0, 15, 3), (0, 15, 1), (0, 14, 4), (0, 14, 0), (0, 12, 4), (0, 13, 5), (3, 12, 1), (2, 11, 1), (2, 12, 0), (1, 10, 1), (1, 11, 0), (0, 9, 1), (0, 10, 0), (4, 13, 2), (3, 14, 2), (3, 13, 3), (3, 13, 1), (2, 15, 2), (2, 14, 3), (2, 14, 1), (2, 13, 4), (2, 13, 0), (1, 16, 2), (1, 15, 3), (1, 15, 1), (1, 14, 4), (1, 14, 0), (1, 12, 4), (1, 13, 5), (0, 17, 2), (0, 16, 3), (0, 16, 1), (0, 15, 4), (0, 15, 0), (0, 14, 5), (0, 11, 4), (0, 12, 5), (0, 13, 6), (4, 12, 1), (3, 11, 1), (3, 12, 0), (2, 10, 1), (2, 11, 0), (1, 9, 1), (1, 10, 0), (0, 8, 1), (0, 9, 0), (5, 13, 2), (4, 14, 2), (4, 13